** **Giới thiệu**

Tạo một báo cáo phân tích về nhân sự để rèn luyện khả năng khám phá và phân tích dữ liệu, tìm hiểu các loại biểu đồ và visualize.

** Mô tả dữ liệu**

 Bảng dữ liệu thể hiện thông tin của nhân viên và các yếu tố quan trọng chủ chốt để hiểu được mức độ nghỉ việc, mức độ hài lòng trong công việc hoặc các đặc điểm khác của nhân viên như:
    + Nhân viên có nghỉ việc hay không
    + Tuổi tác của nhân viên
    + Nhân viên có hay đi công tác hay không
    + Loại phòng ban của nhân viên 
    + Khoảng cách từ nhà đến văn phòng
    + Trình độ học vấn
    + Lĩnh vực học vấn
    + Mức tiền lương hằng giờ
    + Mức độ hài lòng công việc
    + Và nhiều yếu tố khác
    
Mục tiêu là đi khám phá và phân tích dữ liệu để xem sự tương quan giữa các yếu tố, liệu yếu tố nào ảnh hưởng đến sự rời đi của nhân viên và mức độ ảnh hưởng đó ra sao, yếu tố nào ảnh hưởng đến độ hài lòng trong công việc của nhân viên...
        

- Import library và data từ file csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statistics
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

%matplotlib inline


Import dữ liệu

In [ ]:
data = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

- Check size of dataset


In [ ]:
data.shape

Vậy dữ liệu có 1470 index và 35 feature

- Check sơ qua để xem dữ liệu.

In [ ]:
data.head()

In [ ]:
data.columns

- Remove các feature chỉ có 1 giá trị vì sẽ không đóng góp gì đến model

In [ ]:
for i in data.columns:
    x = data[i]
    if len(set(x)) == 1:    
        data.drop(i, axis=1, inplace=True)
len(data.columns)

In [ ]:
data.dtypes

**Check missing value
**

In [ ]:
data.isnull().sum().sum()

OK, there are no missing value. Do đó không cần phải missing value treatment


- Xem thống kê mô tả


In [ ]:
data.describe(percentiles = [.01,.1,.95,.99])

So sánh dữ liệu ở min vs 1% và 99% vs max của 35 feature cho thấy không có sự biến thiên bất thường nên sẽ giữ nguyên bảng dữ liệu.

**Khảo sát về attrition_rate để tìm các yếu tố ảnh hưởng.
**

In [ ]:
data.Attrition.value_counts()

In [ ]:
labels = ['Leave','Stay']
values = [data.Attrition.value_counts().Yes, data.Attrition.value_counts().No]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

Mức độ nghỉ việc của nhân viên thấp so với ở lại

- Chia dữ liệu thành những người ở lại và rời đi để dễ tính toán

In [ ]:
Stay=data[data['Attrition'] == "No"]
Left=data[data['Attrition'] == "Yes"]

                               
          KHẢO SÁT ẢNH HƯỞNG GIỮA BIẾN AGE VÀ ATTRITION
                               

In [ ]:
data["Age"].describe()

In [ ]:
sns.distplot(data['Age']);

Biến Age có phân bố xác xuất khá gần với normal distribution

In [ ]:
data.Age.groupby(data.Attrition).mean()

Bivariate analysis giữa attrition và age

Biểu đồ cho thấy những người trẻ có xu hướng nghỉ việc nhiều hơn.

In [ ]:
sns.distplot(Stay.Age.sample(200), hist = True, kde = True)
sns.distplot(Left.Age.sample(200), hist = True, kde = True)

In [ ]:
sns.boxplot(x="Attrition", y="Age", data=data)

Null Hypothesis : Những người nghỉ việc có độ tuổi tương đương với những người ở lại

Alternate Hypothesis: Những người nghỉ việc có độ tuổi khác với những người ở lại.

In [ ]:
stats.ttest_ind(Stay.Age,Left.Age)

Pvalue < 5% nên reject Null Hypothesis và công nhận Alternate Hypothesis. Vậy độ tuôi là một key factor ảnh hưởng đến attrition rate. Và t value < 0 nên những người ở lại có độ tuổi cao hơn những người rời đi.

     
     KHẢO SÁT GIỮA BIẾN HOURLYRATE VỚI ATTRITION
     

In [ ]:
data.HourlyRate.describe()

So sánh mean giữa 2 group

In [ ]:
data.HourlyRate.groupby(data.Attrition).mean()

In [ ]:
sns.distplot(Stay.HourlyRate, hist = True, kde = True)
sns.distplot(Left.HourlyRate, hist = True, kde = True)

* Việc phân bố của HourlyRate khá giống nhau giữa những người ở lại và rời đi nên đây có thể không là key factor nhưng cũng thông qua t test để đảm bảo chắc chắn

Null Hypothesis : Những người nghỉ việc có mức lương hàng giờ tương đương với những người ở lại

Alternate Hypothesis: Những người nghỉ việc có mức lương hàng giờ khác với những người ở lại.

In [ ]:
stats.ttest_ind(Stay.HourlyRate,Left.HourlyRate)

Pvalue > 5% nên fail to reject Null Hypothesis và reject alternate hypothesis. Vậy mức lương hàng giờ không phải là một key factor ảnh hưởng đến attrition rate.

         
    KHẢO SÁT ẢNH HƯỞNG CỦA DISTANCEFROMHOME VỚI ATTRITION
    

In [ ]:
data.DistanceFromHome.describe()

So sánh mean giữa 2 group ở lại và rời đi thì thấy có sự khác nhau

In [ ]:
data.DistanceFromHome.groupby(data.Attrition).mean()

In [ ]:
sns.distplot(data['DistanceFromHome'])

In [ ]:
# Test 
s1 = pd.Series([])
for i in range(100):
    random_distance_subset = data.DistanceFromHome.sample(n=100)
    s1=pd.concat([s1, random_distance_subset])
sns.distplot(s1)

In [ ]:
sns.catplot(x="Attrition", y="DistanceFromHome",  kind="box", data=data);

In [ ]:
sns.distplot(Stay.DistanceFromHome, hist = True, kde = True)
sns.distplot(Left.DistanceFromHome, hist = True, kde = True)

In [ ]:
d1 = np.log(Stay.DistanceFromHome)
d2 = np.log(Left.DistanceFromHome)

Null Hypothesis : Những người rời đi có khoảng cách từ nhà đến công ty tương đương với những người ở lại

Alternate Hypothesis : Những người rời đi có khoảng cách từ nhà đến công ty khác với những người ở lại


In [ ]:
stats.ttest_ind(d1,d2)

Pvalue < 5% nên reject Null Hypothesis và công nhận Alternate Hypothesis. Vậy DistanceFromHome là một key factor ảnh hưởng đến attrition rate. Và t value < 0 nên những người ở lại có khoảng cách từ nhà đến văn phòng thấp hơn những người rời đi.

- Kiểm thử t test toàn bộ các numeric feature với attrition_rate

In [ ]:
numeric_feature = ['Age','DailyRate','DistanceFromHome','EmployeeNumber','HourlyRate','MonthlyIncome','MonthlyRate','NumCompaniesWorked','PercentSalaryHike','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager']
t_val = []
p_val = []
key_factor = []
for i in numeric_feature:
    t, p = stats.ttest_ind(Stay[i],Left[i])
    t_val.append(t)
    p_val.append(p)
    key_fact = 'No'
    if(p < 0.05):
        key_fact = 'Yes'
    key_factor.append(key_fact)
d = {'name': numeric_feature, 't_val': t_val, 'p_val': p_val, 'Is_keyfactor': key_factor}
df = pd.DataFrame(data=d)
df.sort_values(by=['Is_keyfactor'],ascending=False)

   KHẢO SÁT ẢNH HƯỞNG GIỮA JOBSATISFACTION VỚI ATTRITION

Xem thống kê của attrition với jobsatisfaction

In [ ]:
Att_JbS = pd.crosstab(data['Attrition'],data['JobSatisfaction'])
Att_JbS

In [ ]:
x=['Leave', 'Stay']
fig = go.Figure(go.Bar(x=x, y=[18,27.8], name='Very High'))
fig.add_trace(go.Bar(x=x, y=[19,19.4], name='High'))
fig.add_trace(go.Bar(x=x, y=[30,30.8], name='Medium'))
fig.add_trace(go.Bar(x=x, y=[33,22], name='Low'))
fig.update_layout(barmode='stack')
fig.show()

Null Hypothesis : Những người rời đi có mức độ hài lòng trong công việc tương đương với những người ở lại

Alternate Hypothesis : Những người rời đi có mức độ hài lòng khác với những người ở lại


In [ ]:
chi2, p, dof, ex = stats.chi2_contingency(Att_JbS)
chi2, p

Pvalue < 5% nên reject Null Hypothesis và công nhận Alternate Hypothesis. Vậy JonSatisfaction là một key factor ảnh hưởng đến attrition rate.

   KHẢO SÁT ẢNH HƯỞNG GIỮA JOBLEVEL VỚI ATTRTION

In [ ]:
JL_Att = pd.crosstab(data['JobLevel'],data['Attrition'])
JL_Att

In [ ]:
JL_Att['Yes1'] = (JL_Att['Yes']/ JL_Att['Yes'].sum())*100
JL_Att['No1'] = (JL_Att['No']/ JL_Att['No'].sum())*100
JL_Att

In [ ]:
x=['Stay', 'Leave']
fig = go.Figure(go.Bar(x=x, y=[JL_Att['No1'][1],JL_Att['Yes1'][1]], name=1))
fig.add_trace(go.Bar(x=x, y=[JL_Att['No1'][2],JL_Att['Yes1'][2]], name=2))
fig.add_trace(go.Bar(x=x, y=[JL_Att['No1'][3],JL_Att['Yes1'][3]], name=3))
fig.add_trace(go.Bar(x=x, y=[JL_Att['No1'][4],JL_Att['Yes1'][4]], name=4))
fig.add_trace(go.Bar(x=x, y=[JL_Att['No1'][5],JL_Att['Yes1'][5]], name=5))

fig.update_layout(barmode='stack')
fig.show()

Null Hypothesis : Những người rời đi có JobLevel tương đương với những người ở lại

Alternate Hypothesis : Những người rời đi có JobLevel khác với những người ở lại


In [ ]:
chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(data['Attrition'],data['JobLevel']))
chi2, p

Pvalue < 5% nên reject Null Hypothesis và công nhận Alternate Hypothesis. Vậy JonSatisfaction là một key factor ảnh hưởng đến attrition rate.

Kiểm tra chisquare test cho tất cả categorical feature với attrition

In [ ]:
categorical_feature = ['Attrition','BusinessTravel','Department','Education','EducationField','EnvironmentSatisfaction','Gender','JobInvolvement','JobLevel','JobRole','JobSatisfaction','MaritalStatus','OverTime','PerformanceRating','RelationshipSatisfaction','StockOptionLevel','WorkLifeBalance']
chi2_val = []
p2_val = []
key_factor2 = []
for i in categorical_feature:
    chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(data[i],data['Attrition']))
    chi2_val.append(chi2)
    p2_val.append(p)
    key_fact2 = 'No'
    if(p < 0.05):
        key_fact2 = 'Yes'
    key_factor2.append(key_fact2)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values=['Name', 'Chi2_value','p_val','Is_keyfactor'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[categorical_feature, # 1st column
                       chi2_val,
                       p2_val,
                       key_factor2], # 2nd column
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=1000, height=600)
fig.show()

**Kết luận về attrition rate**:
- Attrition_rate của nhân viên ảnh hưởng đến các yếu tố như độ tuổi, khoảng cách từ nhà đến văn phòng, thu nhập hàng tháng, mức độ hài lòng trong công việc, cấp độ trong công việc...
- Những người có độ tuổi thấp, khoảng cách từ nhà đến văn phòng xa, thu nhập hàng tháng thấp, cấp độ trong công việc thấp và mức độ hài lòng thấp sẽ có xu hướng rời đi...
- Sau khi qua kiểm thử t và chisquare thì attrition ảnh hưởng bởi hầu hết các feature nên đây là 1 biến target.



** Khảo khát JobSatisfaction để tìm các yếu tố ảnh hưởng **

In [ ]:
data.JobSatisfaction.value_counts()

In [ ]:
labels = ['1','2','3','4']

values = [data.JobSatisfaction.value_counts()[1], data.JobSatisfaction.value_counts()[2],data.JobSatisfaction.value_counts()[3],data.JobSatisfaction.value_counts()[4]]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

Mức độ hài lòng của nhân viên cao và rất cao nhiều hơn hẳn mức độ hài lòng ở mức thấp và rất thấp.

Chia dữ liệu để dễ thao tác

In [ ]:
best_satis = data[data['JobSatisfaction'] == 4]
good_satis = data[data['JobSatisfaction'] == 3]
medium_satis= data[data['JobSatisfaction'] == 2]
bad_satis = data[data['JobSatisfaction'] == 1]

   KHẢO SÁT ẢNH HƯỞNG GIỮA DISTANCEFOMHOME VÀ JOBSACTIFACTION

In [ ]:
data.DistanceFromHome.groupby(data['JobSatisfaction']).mean()

In [ ]:
sns.distplot(best_satis.DistanceFromHome, hist = True, kde = True)
sns.distplot(good_satis.DistanceFromHome, hist = True, kde = True)
sns.distplot(medium_satis.DistanceFromHome, hist = True, kde = True)
sns.distplot(bad_satis.DistanceFromHome, hist = True, kde = True)

Null Hypothesis : Khoảng cách từ nhà đến văn phòng của những người có mức độ hài lòng công việc khác nhau thì tương đương nhau.

Alternate Hypothesis : Khoảng cách từ nhà đến văn phòng của những người có mức độ hài lòng công việc khác nhau thì khác nhau.

In [ ]:
stats.f_oneway(best_satis.DistanceFromHome, good_satis.DistanceFromHome, medium_satis.DistanceFromHome, bad_satis.DistanceFromHome)

pvalue > 5% nên fail to reject null hypothesis và reject alternate hypothesis. Vậy khoảng cách từ nhà đến văn phòng không ảnh hưởng đến mức độ hài lòng trong công việc của nhân viên.

   KHẢO SÁT ẢNH HƯỞNG GIỮA DAILYRATE VÀ JOBSACTIFACTION

In [ ]:
data.DailyRate.groupby(data['JobSatisfaction']).mean()

In [ ]:
sns.distplot(best_satis.DailyRate, hist = True, kde = True)
sns.distplot(good_satis.DailyRate, hist = True, kde = True)
sns.distplot(medium_satis.DailyRate, hist = True, kde = True)
sns.distplot(bad_satis.DailyRate, hist = True, kde = True)

Null Hypothesis : Mức lương hàng giờ của những người có mức độ hài lòng công việc khác nhau thì tương đương nhau.

Alternate Hypothesis : Mức lương hàng giờ của những người có mức độ hài lòng công việc khác nhau thì khác nhau.

In [ ]:
stats.f_oneway(best_satis.DailyRate, good_satis.DailyRate, medium_satis.DailyRate, bad_satis.DailyRate)

pvalue > 5% nên fail to reject null hypothesis và reject alternate hypothesis. Vậy mức lương hàng ngày không ảnh hưởng đến mức độ hài lòng trong công việc của nhân viên.

Kiểm thử Anova toàn bộ các numeric feature với job satisfaction

In [ ]:
f_val = []
p_anova = []
anova_key = []
for i in numeric_feature:
    f, p = stats.f_oneway(best_satis[i], good_satis[i], medium_satis[i], bad_satis[i])
    f_val.append(f)
    p_anova.append(p)
    key_fact = 'No'
    if(p < 0.05):
        key_fact = 'Yes'
    anova_key.append(key_fact)
d_anova = {'name': numeric_feature, 'f_val': f_val, 'p_val': p_anova, 'Is_keyfactor': anova_key}
df_anova = pd.DataFrame(data=d_anova)
df_anova

   KHẢO SÁT ẢNH HƯỞNG GIỮA JOBLEVEL VỚI JOBSATISFACTION

In [ ]:
JL_JS = pd.crosstab(data['JobLevel'],data['JobSatisfaction'])
JL_JS

In [ ]:
JL_JS['N1'] = (JL_JS[1]/ JL_JS[1].sum())*100
JL_JS['N2'] = (JL_JS[2]/ JL_JS[2].sum())*100
JL_JS['N3'] = (JL_JS[3]/ JL_JS[3].sum())*100
JL_JS['N4'] = (JL_JS[4]/ JL_JS[4].sum())*100

In [ ]:
x=['Low','Medium','High','Very High']
fig = go.Figure(go.Bar(x=x, y=[JL_JS['N1'][1],JL_JS['N2'][1],JL_JS['N3'][1],JL_JS['N4'][1]], name=1))
fig.add_trace(go.Bar(x=x, y=[JL_JS['N1'][2],JL_JS['N2'][2],JL_JS['N3'][2],JL_JS['N4'][2]], name=2))
fig.add_trace(go.Bar(x=x, y=[JL_JS['N1'][3],JL_JS['N2'][3],JL_JS['N3'][3],JL_JS['N4'][3]], name=3))
fig.add_trace(go.Bar(x=x, y=[JL_JS['N1'][4],JL_JS['N2'][4],JL_JS['N3'][4],JL_JS['N4'][4]], name=4))
fig.add_trace(go.Bar(x=x, y=[JL_JS['N1'][5],JL_JS['N2'][5],JL_JS['N3'][5],JL_JS['N4'][5]], name=5))

fig.update_layout(barmode='stack')
fig.show()

Job level khá tương đồng giữa những nhóm người có mức độ hài lòng trong công việc giống nhau.

Null Hypothesis : Job level của những người có mức độ hài lòng công việc khác nhau thì tương đương nhau.

Alternate Hypothesis : Job level của những người có mức độ hài lòng công việc khác nhau thì khác nhau.

In [ ]:
chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(data['JobLevel'],data['JobSatisfaction']))
chi2, p

pvalue > 5% nên fail to reject null hypothesis và reject alternate hypothesis. Vậy job level không ảnh hưởng đến mức độ hài lòng trong công việc của nhân viên.

   KHẢO SÁT ẢNH HƯỞNG GIỮA BUSINESS TRAVEL VỚI JOB SATISFACTION

In [ ]:
BT_JS = pd.crosstab(data['BusinessTravel'],data['JobSatisfaction'])
BT_JS

In [ ]:
BT_JS['N1'] = (BT_JS[1]/ BT_JS[1].sum())*100
BT_JS['N2'] = (BT_JS[2]/ BT_JS[2].sum())*100
BT_JS['N3'] = (BT_JS[3]/ BT_JS[3].sum())*100
BT_JS['N4'] = (BT_JS[4]/ BT_JS[4].sum())*100
BT_JS
# BT_JS['N1'][0]

In [ ]:
x=['Low','Medium','High','Very High']
fig = go.Figure(go.Bar(x=x, y=[BT_JS['N1'][0],BT_JS['N2'][0],BT_JS['N3'][0],BT_JS['N4'][0]], name='Non-Travel'))
fig.add_trace(go.Bar(x=x, y=[BT_JS['N1'][1],BT_JS['N2'][1],BT_JS['N3'][1],BT_JS['N4'][1]], name='Travel_frequently'))
fig.add_trace(go.Bar(x=x, y=[BT_JS['N1'][2],BT_JS['N2'][2],BT_JS['N3'][2],BT_JS['N4'][2]], name='Travel_Rarely'))

fig.update_layout(barmode='stack')
fig.show()

Job level khá tương đồng giữa những nhóm người có mức độ hài lòng trong công việc giống nhau.

Null Hypothesis : Business Travel của những người có mức độ hài lòng công việc khác nhau thì tương đương nhau.

Alternate Hypothesis : Business Travel của những người có mức độ hài lòng công việc khác nhau thì khác nhau.

In [ ]:
chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(data['BusinessTravel'],data['JobSatisfaction']))
chi2, p

pvalue > 5% nên fail to reject null hypothesis và reject alternate hypothesis. Vậy business travel không ảnh hưởng đến mức độ hài lòng trong công việc của nhân viên.

Kiểm thử chisquare cho tất cả categorical feature với job satisfaction

In [ ]:
chi2_val_js = []
p2_val_js = []
key_factor2_js = []
for i in categorical_feature:
    chi2, p, dof, ex = stats.chi2_contingency(pd.crosstab(data[i],data['JobSatisfaction']))
    chi2_val_js.append(chi2)
    p2_val_js.append(p)
    key_fact2_js = 'No'
    if(p < 0.05):
        key_fact2_js = 'Yes'
    key_factor2_js.append(key_fact2_js)


In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['Name', 'Chi2_value','p_val','Is_keyfactor'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[categorical_feature, # 1st column
                       chi2_val_js,
                       p2_val_js,
                       key_factor2_js], # 2nd column
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=1000, height=600)
fig.show()

**Kết luận với jobsatisfaction**
- Jobsatisfaction thì dường như không bị ảnh hưởng bởi bất kì các yếu tố khác nào ngoại trừ hourlyrate nên feature này là một biến độc lập.


**Kết luận chung**: qua bài tập này đã sử dụng pair simple t test, ANOVA và chi square để tìm sự tương quan giữa các biến, iểu được có hay không sự ảnh hưởng của 1 biến với target. Tìm hiểu về các thư viện như matplotlib, seaborn, plotly và các dạng biểu đồ để trực quan hóa được dữ liệu.